# Potential

The electron beam interacts with the specimen through the Coulomb potential of the electrons and nuclei in the specimen. 

## Potential parametrization

The electron charge distribution of an atom can be calculated from a first-principles electronic structure calculation, while the atomic nuclei are a point charges at the resolution of a TEM. Given a charge distribution, the potential can be obtained via Poisson's equation. 

Simulation codes include a parametrization of the atomic potentials, with a table of parameters for each element fitted to Hartree-Fock calculations. Below we show the radial depedence of the electrostatic potential of five selected elements, using the default parametrization in abTEM. This parametrization is due to Lobato et. al., it is the latest and one of the most accurate parametrizations.

In [1]:
# This cell is hidden from nbsphinx

from abtem.parametrizations import lobato, load_lobato_parameters
import matplotlib.pyplot as plt
from ase.data import chemical_symbols
import numpy as np

parameters = load_lobato_parameters()
atomic_numbers = [6, 14, 29, 79, 92]
r = np.linspace(0,2,100)

plt.figure(figsize=(5,3.5))
for atomic_number in atomic_numbers:
    v = lobato(r, parameters[atomic_number])
    plt.plot(r, v, label=chemical_symbols[atomic_number])

plt.xlabel('r [Å]')
plt.ylabel('V [eV / e]')
plt.ylim([0, 30])
plt.legend()
plt.tight_layout()
plt.savefig('potentials.png')
plt.close()

The radial dependence of the atomic potential for isolated single atoms is shown below.

![potentials](potentials.png)

## Independent atom model

The full specimen potential, $V(r)$, is usually obtained as a linear superposition of the atomic potentials

$$
    V(r) = \sum_i V_i(r-r_i) \quad ,
$$

where $V_i(r)$ is the atomic potential of the $i$'th atom. This is known as the Indenpendent Atom Model (IAM). This model obviously neglects any effect due to bonding and charge transfer. However, since the nucleus and core electrons constitute most of the charge in an atom, this is often a good approximation, which gets progressively better as the atomic number increases. Nonetheless, due to recent improvements in microscopes, interest in going beyond this approximation have started. For example, it has been shown that simulated images based on potentials from DFT calculations can provide a better match with experimental data for light elements. In a later tutorial, we go beyond the IAM using density functional theory.

Below we create a `Potential` object. This object represents a sliced potential in the IAM using a given parametrization. 

In [3]:
from ase.io import read
from abtem.potentials import Potential

srtio = read('srtio3_110.cif')

potential = Potential(srtio, slice_thickness=0.5, sampling=.05, parametrization='lobato')

In [5]:
# This cell is hidden from nbsphinx

from abtem.plot import plot_image

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9, 3.2))
plot_image(potential.get_slice(3), ax=ax1, title='Slice # 3')
plot_image(potential.get_slice(5), ax=ax2, title='Slice # 5')
plot_image(potential.get_slice(7), ax=ax3, title='Slice # 7')
plt.tight_layout()
plt.savefig('potential_slices.png')
plt.close()

The `get_slice` method of the `Potential` object can be used to calculate individual slices. Below we show three slices from the SrTiO<sub>3</sub> potential.

![potential_slices](potential_slices.png)

## Potential projection

Some multislice codes uses a fast, but somewhat less accurate approximation for slicing the potential. The atomic coordinates are sorted into slices and the analytical infinite projection of the atomic potentials are assigned to that slice. This means that slices cannot be made thinner than about $1-2$ Å, and they should preferably coincide with lattice planes. This method can be used in abTEM by setting `method = 'infinite'` when instantiating the potential. By default, abTEM uses the slower but more accurate finite potential projection. Here, numerical integration is used to calculate the integrals of the form

$$
V_{proj}^{(i)}(x, y) = \int_{z_i}^{z_i+\Delta z} V(x,y,z) dz \quad ,
$$

where $z_i$ is the $z$-position at the entrance of the $i$'th slice and $\Delta z$ is the slice thickness. Using this method, the slices can be arbitrarily thin, however, a slice thickness of $0.5$ Å is generally slim enough. The numerical integrals are efficiently handled by using the double exponential Tanh–Sinh quadrature, which is designed accurate result using a minimum number evaluations for functions with singularities.